# Линейная регрессия с L2 регуляризацией

Введение
Линейные методы хорошо подходят для работы с разреженными данными — к таковым относятся, например, тексты. Это можно объяснить высокой скоростью обучения и небольшим количеством параметров, благодаря чему удается избежать переобучения.

Линейная регрессия имеет несколько разновидностей в зависимости от того, какой регуляризатор используется. Мы будем работать с гребневой регрессией, где применяется квадратичный, или L2-регуляризатор.

Для извлечения TF-IDF-признаков из текстов воспользуйтесь классом sklearn.feature_extraction.text.TfidfVectorizer.

Для предсказания целевой переменной мы будем использовать гребневую регрессию, которая реализована в классе sklearn.linear_model.Ridge.

Обратите внимание, что признаки LocationNormalized и ContractTime являются строковыми, и поэтому с ними нельзя работать напрямую. Такие нечисловые признаки с неупорядоченными значениями называют категориальными или номинальными. Типичный подход к их обработке — кодирование категориального признака с m возможными значениями с помощью m бинарных признаков. Каждый бинарный признак соответствует одному из возможных значений категориального признака и является индикатором того, что на данном объекте он принимает данное значение. Данный подход иногда называют one-hot-кодированием. Он уже реализован в классе sklearn.feature_extraction.DictVectorizer. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn; seaborn.set()

In [26]:
data_train = pd.read_csv('salary-train.csv')
data_test = pd.read_csv('salary-test-mini.csv')
data_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [27]:
# Приведите тексты к нижнему регистру (text.lower())
data_train['FullDescription'] = data_train['FullDescription'].map(lambda x: x.lower())
data_test['FullDescription'] = data_test['FullDescription'].map(lambda x: x.lower())
# Замените все, кроме букв и цифр, на пробелы
data_train['FullDescription'] = data_train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
data_test['FullDescription'] = data_test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [28]:
data_test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,Milton Keynes,contract,NaN
1,a web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [29]:
# Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, 
# которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer)
from sklearn.feature_extraction.text import TfidfVectorizer
TF_IDF = TfidfVectorizer(min_df=5)
X_descr_train = TF_IDF.fit_transform(data_train['FullDescription'])
X_descr_test = TF_IDF.transform(data_test['FullDescription'])

In [20]:
print np.any(data_train['LocationNormalized'].isnull())
print np.any(data_train['ContractTime'].isnull())

False
True


In [30]:
# Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'.
data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)
data_test['LocationNormalized'].fillna('nan', inplace=True)
data_test['ContractTime'].fillna('nan', inplace=True)

In [43]:
# Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [44]:
print X_descr_train.shape
print X_train_categ.shape
print X_descr_test.shape
print X_test_categ.shape

(60000, 22861)
(60000, 1766)
(2, 22861)
(2, 1766)


In [45]:
# Объедините все полученные признаки в одну матрицу "объекты-признаки
from scipy.sparse import hstack
Xtrain = hstack([X_descr_train, X_train_categ])
Xtest = hstack([X_descr_test, X_test_categ])

In [46]:
print Xtrain.shape
print Xtest.shape

(60000L, 24627L)
(2L, 24627L)


In [36]:
ytrain = data_train['SalaryNormalized']

In [47]:
from sklearn.linear_model import Ridge
model = Ridge(alpha=1, random_state=241)
model.fit(X=Xtrain, y=ytrain)
ypred = model.predict(X=Xtest)

In [48]:
print ypred

[56555.61500155 37188.32442618]


In [49]:
p = str(np.round(ypred[0],2)) +' '+str(np.round(ypred[1],2))
print p

56555.62 37188.32


In [50]:
f = open('salary.txt', 'w')
f.write(p)
f.close()